In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.17.2"
#r "nuget: SkiaSharp"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.6.3-alpha"
#r "nuget: System.Linq.Async"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.ChatCompletion;


In [2]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var settings  = Settings.LoadFromFile("config/settings.json");
#pragma warning disable SKEXP0001, SKEXP0020, SKEXP0010

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

__Chunking__

Split the data into smaller pieces. In this case paragraphs.

In [3]:
using System.IO;

var filename = "data/ai search/poem.txt";
var poemParagraphs = File.ReadAllText(filename).
Split(new string[] { "\r\n\r\n", "\n\n" }, StringSplitOptions.RemoveEmptyEntries);


__Adding data to the index__

For the data storage part an Azure AI Search service is used.

In [4]:
using System.Collections;
using System.Linq;
using System.Text.Json;

var collectionName = "PoenIndex";

#pragma warning disable SKEXP0001, SKEXP0020, SKEXP0010
var memory = new MemoryBuilder()
                    .WithAzureOpenAITextEmbeddingGeneration(settings.textembeddingmodel, settings.azureEndpoint, settings.apiKey)
                    .WithMemoryStore(new AzureAISearchMemoryStore(settings.aisearchendpoint, settings.aisearchkey))
                    .Build();


foreach (var paragraph in poemParagraphs)
{
    await memory.SaveInformationAsync(
        collection: collectionName,
        text: paragraph,
        id: paragraph.GetHashCode().ToString(),
        description: paragraph,
        additionalMetadata: JsonSerializer.Serialize(new { filename })

            );
}

In [5]:
var systemMessage = 
"""
You are a helpfull asssitant.
You help users getting answers on their questions.
Please also explain why you think the answer is correct and how you came up with it.
""";

var chat = new ChatHistory(systemMessage);

string GetUserMessage(List<string> messages, string question)
{
    var sb = new StringBuilder();
    sb.AppendLine("Kindly utilize the discovered results to respond to the provided question.");
    sb.AppendLine("# Question");
    sb.AppendLine(question);
    sb.AppendLine("# Found Results:");
    for(int i = 0; i<messages.Count; i++) 
    {
        sb.AppendLine($"- {i+1}");
        sb.AppendLine(messages[i]);
    }
    return sb.ToString();
}


In [6]:
var question = new[]{"Did the Unicorn got captured?", "What spell has the developer woven?"};
foreach(var q in question)
{
    Console.WriteLine($"###");
    Console.WriteLine($"Question:");
    Console.WriteLine($"{q}");
    var results = await memory.SearchAsync(collectionName, q, limit: 2).ToListAsync();
    var textResult = results.Select(r => r.Metadata.Description).ToList();
    
    chat.AddUserMessage(GetUserMessage(textResult, q));

    var assistantReply = await chatGPT.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings(){
        Temperature = 0.7
    });
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"Answer:");
    Console.WriteLine($"{assistantReply}");
    Console.WriteLine($"");
}